## Model Deployment with Streamlit and Localtunnel <a name="s4"></a>

[Streamlit](https://streamlit.io/) is an open-source app framework for Machine Learning and Data Science teams. It turns data scripts into shareable web apps in minutes. 

To deploy my model with streamlit, I create a python file `app.py` which contains the following code:

```
from transformers import *
from summarizer import Summarizer
from summarizer.text_processors.coreference_handler import CoreferenceHandler
import streamlit as st

st.title('Extractive and Abstractive Text Summarization')
st.markdown('Using BERT and BART Transformer Models')

text = st.text_area('Please Input a Long Scientific Text')
abstract = st.text_area("Please Input Scientific Text Abstract")
conclusion = st.text_area("Please Input Scientific Text Conclusion")

pretrained_model = 'allenai/scibert_scivocab_uncased'

max_length = 750
min_length = 250

@st.cache(suppress_st_warning=True)
def get_summary(text, abstract, conclusion, pretrained_model):
    # Extractive Summarizer
    # Load model, model config and tokenizer via Transformers
    custom_config = AutoConfig.from_pretrained(pretrained_model)
    custom_config.output_hidden_states=True
    custom_tokenizer = AutoTokenizer.from_pretrained(pretrained_model)
    custom_model = AutoModel.from_pretrained(pretrained_model, config=custom_config)

    # Create pretrained-model object
    extractive_model = Summarizer(custom_model=custom_model, custom_tokenizer=custom_tokenizer)

    # Abstractive Summarizer
    abstractive_summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

    optimal_num_sentences = extractive_model.calculate_optimal_k(text, k_max=10)
    extractive_summarized_text = "".join(extractive_model(text, num_sentences=optimal_num_sentences))
    
    text_list = [abstract, extractive_summarized_text, conclusion]
    joined_text = " ".join(text_list)

    abstractive_summary = abstractive_summarizer(joined_text, max_length=max_length, min_length=min_length, 
                                                do_sample=False)[-1]["summary_text"]
    st.write("Summary")
    st.success(abstractive_summary)

if st.button("Summarize"):
    get_summary(text, abstract, conclusion, pretrained_model)
```

The code above in `app.py` performs the following:
1. Creates a streamlit text area for the user to place a long scientific text document.
2. Create a `Summarize` button for the user to click.
3. Once `Summarize` is click, the model performs extractive summarization and then abstractive summarization. The result is presented to the user. 

In order for this to be shared through the web, I make use of [localtunnel](https://github.com/localtunnel/localtunnel). Localtunnel exposes your localhost to the world for easy testing and sharing! No need to mess with DNS or deploy just to have others test out your changes. It is great for working with browser testing tools like browserling or external api callback services like twilio which require a public url for callbacks.

With localtunnel, I am able to deploy the streamlit interface to the web. 

To achieve all this, first ensure `app.py` is in the same directory as this notebook. Then, run the codeblocks below. 

In [ ]:
!pip install streamlit

     |████████████████████████████████| 10.1 MB 5.4 MB/s 
     |████████████████████████████████| 4.3 MB 34.6 MB/s 
     |████████████████████████████████| 164 kB 48.9 MB/s 
     |████████████████████████████████| 111 kB 45.8 MB/s 
     |████████████████████████████████| 181 kB 46.2 MB/s 
     |████████████████████████████████| 76 kB 5.4 MB/s 
     |████████████████████████████████| 63 kB 1.3 MB/s 
     |████████████████████████████████| 130 kB 47.6 MB/s 
     |████████████████████████████████| 793 kB 40.8 MB/s 
     |████████████████████████████████| 380 kB 47.9 MB/s 
  Created wheel for blinker: filename=blinker-1.4-py3-none-any.whl size=13478 sha256=5da4a23ca2a6bf867307c5f667618075f750848be23fc9b1fec10a0616deaac1
  Stored in directory: /root/.cache/pip/wheels/22/f5/18/df711b66eb25b21325c132757d4314db9ac5e8dabeaf196eab
Successfully built blinker
  Attempting uninstall: prompt-toolkit
    Found existing installation: prompt-toolkit 1.0.18
    Uninstalling prompt-toolkit-1.0.18:
      

In [ ]:
!streamlit run app.py & npx localtunnel --port 8501

2022-03-26 15:44:56.167 INFO    numexpr.utils: NumExpr defaulting to 2 threads.
npx: installed 22 in 4.559s

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://34.68.157.228:8501

your url is: https://calm-hound-63.loca.lt
2022-03-26 15:45:30.922080: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: spacy.morphology.Morphology size changed, may indicate binary incompatibility. Expected 104 from C header, got 112 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: spacy.vocab.Vocab size changed, may indicate binary incompatibility. Expected 96 from C header, got 104 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: sp

These codeblocks will expose your work to the web. It will return a URL for you to share in order to access the streamlit model.